### Extract from OSM
The purpose of this script is to show how to go from a .osm.pbf file to a network format that GOSTnets will accept. 
Additional modifications in this script include: clipping the roads to an input polygon (here, SL_adm0.shp), and slicing the roads DataFrame by a list of accepted road types, as tagged in OSM (see accepted_road_types list). 

In [4]:
import geopandas as gpd
import pandas as pd
import os, sys
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
import GOSTnet as gn
import LoadOSM as losm
import importlib

Define filepaths

In [10]:
pth = r'C:\Users\charl\Documents\GOST\SierraLeone\Temp'
fil = r'sierra-leone-latest.osm.pbf'
f = os.path.join(pth, fil)

Using the loadOSM library, pass the filepath of the .osm.pbf to initiate a OSM_to_network class

In [12]:
sierra_leone_rn = losm.OSM_to_network(f)

This generates a property of the class called 'roads_raw'. This is a raw dataframe of the highway objects, extracted from the .osm.pbf. This is the starting point for our network

In [13]:
sierra_leone_rn.roads_raw.infra_type.value_counts()

path              47484
track             17025
unclassified      13833
residential       13546
footway            1722
service            1358
tertiary           1013
secondary           536
primary             440
trunk               266
primary_link         38
secondary_link       16
trunk_link           13
pedestrian           12
bridleway             8
stream                7
construction          7
tertiary_link         5
yes                   3
triver                2
river                 2
living_street         1
Name: infra_type, dtype: int64

In [14]:
accepted_road_types = ['motorway','motorway_link','residential', 'unclassified', 'track','service','tertiary','road','secondary','primary','trunk','primary_link','trunk_link','tertiary_link','secondary_link']

We call the filerRoads method and pass it a list of acceptable road types

In [15]:
sierra_leone_rn.filterRoads(acceptedRoads = accepted_road_types)

In [16]:
sierra_leone_rn.roads_raw.infra_type.value_counts()

track             17025
unclassified      13833
residential       13546
service            1358
tertiary           1013
secondary           536
primary             440
trunk               266
primary_link         38
secondary_link       16
trunk_link           13
tertiary_link         5
Name: infra_type, dtype: int64

Here, we import a shapefile for our AOI (Sierra Leone country bound) and clip to its extent using standard GeoPandas functions

In [19]:
util_pth = r'C:\Users\charl\Documents\GOST\SierraLeone\Util_files'
shp = gpd.read_file(os.path.join(util_pth, 'SL_bound.shp'))
shp = shp.to_crs({'init':'epsg:4326'})
shp_obj = shp.geometry.iloc[0]

In [20]:
sierra_leone_rn.roads_raw = sierra_leone_rn.roads_raw.loc[sierra_leone_rn.roads_raw.geometry.intersects(shp_obj) == True]

Having narrowed the DataFrame to the roads we are interested in, we generate the RoadsGDF object

In [21]:
sierra_leone_rn.generateRoadsGDF(verbose = False)

With initialReadIn(), we transform this to a graph object

In [22]:
sierra_leone_rn.initialReadIn()

We save this graph object down to file using gn.save()

In [24]:
gn.save(sierra_leone_rn.network,'SL_uncleaned',r'C:\Users\charl\Documents\GOST\SierraLeone\RoadNet')